In [1]:
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests
import re

In [2]:
brand_list = ['audi', 'skoda', 'honda', 'volvo', 'bmw', 'nissan',
              'infiniti', 'toyota', 'lexus', 'volkswagen', 'mitsubishi', 'mercedes'] #список брендов из тестового датасета

In [ ]:
df = pd.DataFrame()

In [ ]:
# запускаем цикл парсинга для каждого бренда из списка
for b in brand_list:
    
    headers={'User-Agent': 'Mozilla/5.0'}
    # получаем начальную страницу для всех объявлений бренда
    response = requests.get('https://auto.ru/cars/'+b+'/used/', headers={'User-Agent': 'Mozilla/5.0'})
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    # находим сколько всего страниц с объявлениями данного бренда
    amount_pages = int(soup.find_all('div', class_='ListingCarsPagination')[0].find_all('a', class_='Button')[-3].get_text())
    # определим класс для удобства записи и хранения некоторой информации об авто
    class CarCard():
        pass

    # запускаем цикл перебора страниц объявлений для конкретного бренда
    for n in range(0, amount_pages):
        links_to_cars = [] # создадим список в котором будут храниться ссылки на все авто данной страницы
        soup = BeautifulSoup(response.text, 'html.parser')
        list_audi = soup.find_all('div', class_='ListingItem__main') # получаем список из объявлений на текущей странице цикла
        car_card = CarCard()
    
        # запускаем цикл по всем объявлениям на текущей странице
        for i in range(0, len(list_audi)):
            car_card.url = list_audi[i].find('a', class_='ListingItemTitle__link')['href'] # находим на странице ссылку на каждое авто
            # находим на странице цену для каждого авто
            car_card.price = list_audi[i].find('div', class_='ListingItemPrice__content').get_text().replace('\xa0', ' ') if list_audi[i].find('div', class_='ListingItemPrice__content') is not None else 'sold'
            links_to_cars.append([car_card.url, car_card.price]) # добавляем в ранее определенный список ссылку на авто и его цену
        # запускаем цикл со ссылками на авто текущей страницы
        for f,s in links_to_cars:
            print('start car', f)
            during_car_dict = {} # создаем словарь в который будем записывать всю информацию по текущему авто 
            res = requests.get(f, headers={'User-Agent': 'Mozilla/5.0'})
            res.encoding = 'utf-8'
            car = BeautifulSoup(res.text, 'html.parser')
            if 'Ошибка' in car.title.text: # пропускаем текущее авто если страница не открылась
                continue
            information = car.find('ul', class_='CardInfo') # получаем основную информацию об авто из объявления
            link_to_catalog = car.find('a', class_='CardCatalogLink') # получаем ссылку на каталог данного авто для доп. информации, если это возможно
            if link_to_catalog is not None:
                link_to_catalog = link_to_catalog['href']
                res_catalog = requests.get(link_to_catalog, headers=headers)
                res_catalog.encoding = 'utf-8'
                catalog_inf = BeautifulSoup(res_catalog.text, 'html.parser')
                # если удается открыть каталог, получаем информацию о кол-ве дверей в авто и годах выпуска данной модели
                if 'Ошибка' not in catalog_inf.title.text:
                    doors = catalog_inf.find('dt', class_='list-values__label', string='Количество дверей')
                    doors_amount = int(doors.find_next_sibling('dd').text)
                    model_year = catalog_inf.find('a', class_='search-form-v2-mmm__breadcrumbs-item_type_generation').text
                    during_car_dict['doors_amount'] = doors_amount
                    during_car_dict['model_year'] = model_year
            during_car_dict['model'] = car.find_all('h1', class_='CardHead__title')[0].get_text()
            during_car_dict['url'] = f
            during_car_dict['price'] = s
            during_car_dict['parsing_unixtime'] = round(time.time())
        
            for i in information:
                test_param = i.get_text(',').replace('\xa0', ' ').split(',', 1)
                during_car_dict[test_param[0]] = test_param[1]
            
            # создаем датасет с характеристиками одного авто и объединяем с существующим датасетом
            mini_df = pd.DataFrame(data=during_car_dict, index=range(0,1)) 
            df = pd.concat([df, mini_df], ignore_index=True)    
    
        if n == amount_pages:
            continue
        # получаем ссылку на следующую страницу с объявлениями и продолжаем цикл
        next_page = soup.find_all('div', class_='ListingCarsPagination')[0].find_all('a', class_='Button')[-1]['href']
        response = requests.get(next_page, headers={'User-Agent': 'Mozilla/5.0'})
        response.encoding = 'utf-8'

## Далее немного обработаем данные полученые при парсинге, и сразу выделим из них пару отдельных признаков

In [ ]:
df['brand'] = df['model'].apply(lambda x: x.split(' ')[0])
df['Двигатель'] = df['Двигатель'].apply(lambda x: x.split('/'))
df['fuel_type'] = df['Двигатель'].apply(lambda x: x[2][2:])
df['fuel_type'] = df['fuel_type'].apply(lambda x: 'Электро' if x=='лектро' else x)
df['enginePower'] = df['Двигатель'].apply(lambda x: x[1])
df['engineVolume'] = df['Двигатель'].apply(lambda x: x[0])
pattern = re.compile('20\d\d|19\d\d')
df['modelDate'] = df['model_year'].apply(lambda x: pattern.findall(x)[0]if type(x) is str else x)
df['modelDate'] = df['modelDate'].apply(float)

## Записываем получившийся датасет в файл

In [ ]:
df.to_csv('data_auto.csv', index=False)